## Preprocessing

#### Using Buckwalter translitration
- o سكون
- a فتحة
- i كسرة
- ~ شدة
- u ضمة
- N تنوين بالضم
- F تنوين بالفتح
- K تنوين بالكسر

In [56]:
# Imports
import re
from lang_trans.arabic import buckwalter
from pyarabic.araby import tokenize, is_arabicrange

### Constants

In [57]:
DIACRITICS = ['o', 'a', 'i', '~', 'u', 'N', 'F', 'K', ' ']
print(len(DIACRITICS) + len(["~a", "~i", "~u", "~N", "~F", "~K"]))

15


### Word Level Preprocessing

In [58]:
def remove_undesired_characters(sentence):
    # Except: . ? ! ,
    pattern = re.compile('[\[\]\\/@#\$&%\^\+<=>«»–(){}\*\|\`:;\'"\~_!\.\?؟\,،؛-]')
    return re.sub(pattern, '', " ".join(tokenize(sentence, conditions=is_arabicrange)))

def remove_html_tags(sentence):
    pattern = re.compile('<.*?>')
    return re.sub(pattern, '', sentence)

def remove_numbers(sentence):
    pattern = re.compile('[٠-٩0-9]')
    return re.sub(pattern, '', sentence)

def remove_english_letters(sentence):
    pattern = re.compile('[a-zA-Z]')
    return re.sub(pattern, '', sentence)

def data_cleaning(sentence):
    sentence = remove_english_letters(sentence)
    sentence = remove_numbers(sentence)
    sentence = remove_html_tags(sentence)
    sentence = remove_undesired_characters(sentence)
    
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = ' '.join(sentence.split())
    return sentence

def tokenization(sentence):
    # tokenizer = Tokenizer(num_words=None,
    # filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
    # lower = False, split = ' ')
    return sentence.split()

def word_level_preprocess(sentence):
    cleanedSentence = data_cleaning(sentence)
    finalSentence = tokenization(cleanedSentence)
    return finalSentence

### Buckwalter

In [59]:
def run_buckwalter(sentence):
    sentence = word_level_preprocess(sentence)
    output = []
    for word in sentence:
        output.append(buckwalter.transliterate(word))
    return output

def to_arabic(sentence):
    output = []
    for word in sentence:
        output.append(buckwalter.untransliterate(word))
    return output

### Remove Diacritics

In [60]:
# Function used to divide each word to the following tuple
# (each char of the word, its corresponding diacritic)
def extract_labels(sentences):
    output_char = []
    output_diacritics = []

    for word in sentences:
        diacritics = []
        word_char = []
        skip = False


        for i in range(len(word)):
            if not skip:
                if word[i] not in DIACRITICS:
                    word_char.append(word[i])
                    if i == len(word) - 1 or (i + 1 < len(word) and word[i + 1] not in DIACRITICS):
                        diacritics.append(" ")
                else:
                    if word[i] == "~":
                        if i + 1 < len(word) and word[i + 1] in DIACRITICS:
                            diacritics.append(f"~{word[i + 1]}")
                            skip = True
                    else:
                        diacritics.append(word[i])
            else: skip = False
        
        output_char.append(word_char)
        output_diacritics.append(diacritics)

    return output_char, output_diacritics

### Main

In [61]:

# sentence = "( قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ ) أَيْ الْوَصِيَّةُ ( قَوْلُهُ مَا مَرَّ ) أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَبْطُلُ ، وَعِبَارَةُ الْكَنْزِ وَلَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي لَمْ يَتَعَيَّنْ غَنَمُهُ إنْ كَانَتْ انْتَهَتْ ا ه سم ( قَوْلُهُ فَيُعْطَى وَاحِدَةً مِنْهَا إلَخْ ) كَمَا لَوْ كَانَتْ مَوْجُودَةً عِنْدَ الْوَصِيَّةِ وَالْمَوْتِ ، وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ فِي الصُّورَتَيْنِ وَإِنْ تَرَاضَيَا ؛ لِأَنَّهُ صُلْحٌ عَلَى مَجْهُولٍ مُغْنِي وَنِهَايَةٌ قَالَ ع ش قَوْلُهُ وَاحِدَةً مِنْهَا أَيْ كَامِلَةً ، وَلَا يَجُوزُ أَنْ يُعْطَى نِصْفَيْنِ مِنْ شَاتَيْنِ ؛ لِأَنَّهُ لَا يُسَمَّى شَاةً وَقَوْلُهُ وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ وَيَنْبَغِي أَنْ يُقَالَ مِثْلُ ذَلِكَ فِي الْأَرِقَّاءِ ا ه ."
# sentence = "( وَلَا يَجُوزُ أَنْ يَصُومَ عَنْ بَعْضِ الْجَزَاءِ ، وَيُطْعِمَ عَنْ بَعْضٍ ) ، نَصَّ عَلَيْهِ ، لِأَنَّهَا كَفَّارَةٌ وَاحِدَةٌ ، فَلَمْ يَجُزْ فِيهَا كَسَائِرِ الْكَفَّارَاتِ .( 6 / 189 )"
sentence = "1180 – مَسْأَلَةٌ - وَمَنْ حَنِثَ وَهُوَ قَادِرٌ عَلَى الإِطْعَامِ، أَوْ الْكِسْوَةِ، أَوْ الْعِتْقِ، ثُمَّ افْتَقَرَ فَعَجَزَ، عَنْ كُلِّ ذَلِكَ: لَمْ يُجْزِهِ الصَّوْمُ أَصْلًا"
after_buckwalter = run_buckwalter(sentence)

char_list, diacritics_list = extract_labels(after_buckwalter)
print("Char", char_list)
print("Diac", diacritics_list)

Char [['m', 's', '>', 'l', 'p'], ['w', 'm', 'n'], ['H', 'n', 'v'], ['w', 'h', 'w'], ['q', 'A', 'd', 'r'], ['E', 'l', 'Y'], ['A', 'l', '<', 'T', 'E', 'A', 'm'], ['>', 'w'], ['A', 'l', 'k', 's', 'w', 'p'], ['>', 'w'], ['A', 'l', 'E', 't', 'q'], ['v', 'm'], ['A', 'f', 't', 'q', 'r'], ['f', 'E', 'j', 'z'], ['E', 'n'], ['k', 'l'], ['*', 'l', 'k'], ['l', 'm'], ['y', 'j', 'z', 'h'], ['A', 'l', 'S', 'w', 'm'], ['>', 'S', 'l', 'A']]
Diac [['a', 'o', 'a', 'a', 'N'], ['a', 'a', 'o'], ['a', 'i', 'a'], ['a', 'u', 'a'], ['a', ' ', 'i', 'N'], ['a', 'a', ' '], [' ', ' ', 'i', 'o', 'a', ' ', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'a', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'i'], ['u', '~a'], [' ', 'o', 'a', 'a', 'a'], ['a', 'a', 'a', 'a'], ['a', 'o'], ['u', '~i'], ['a', 'i', 'a'], ['a', 'o'], ['u', 'o', 'i', 'i'], [' ', ' ', '~a', 'o', 'u'], ['a', 'o', 'F', ' ']]
